In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import load_model
from pickle import load
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


In [2]:
#reading pre trained model for image caption generation

max_length = 32
tokenizer = load(open("tokenizer.p","rb"))
model = load_model('models/model_0.h5')
xception_model = Xception(include_top=False, pooling="avg")


In [3]:
#Function to help generate caption from image

def extract_features(filename, model):
        try:
            image = Image.open(filename)
            
        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None


def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text



In [4]:
# Generate Caption
def generate_caption():
    global x,l

    photo = extract_features(x, xception_model)
    img = Image.open(x)
    description = generate_desc(model, tokenizer, photo, max_length)
    description = ' '.join(description.split(' ')[:-1])
    description = ' '.join(description.split(' ')[1:])
    print(description)
    
    l=Label(root,text="-----------------------------------------------------------------------")
    l.place(x = 80,y = 370) 
    l=Label(root,text="{}".format(description),font="bold 9 roman")
    l.place(x = 100,y = 370) 
    
def delete_caption():
    global l
    del l
    l=Label(root,text="-----------------------------------------------------------------------")
    l.place(x = 80,y = 370) 

In [5]:
#Open New File

def openfilename(): 
    global img_lcn_arr,count_img
    filename = filedialog.askopenfilename(title ='"Select the image') 
    img_lcn_arr.append(filename)
    count_img=len(img_lcn_arr)
    return filename

def open_img():
    global root,x, IMG_H,IMG_W
    x = openfilename() 
    img = Image.open(x) 
    img = img.resize((IMG_H, IMG_W), Image.ANTIALIAS) 
    img = ImageTk.PhotoImage(img) 
    panel = Label(root, image = img) 
    panel.image = img 
    panel.place(x=100, y=60)

In [6]:
# Resize Image
def open_img_zoom():
    global root,x, IMG_H,IMG_W

    img = Image.new('RGB', (300,300), (242, 242, 242))
    img = ImageTk.PhotoImage(img) 
    panel = Label(root, image = img) 
    panel.image = img 
    panel.place(x=100, y=60)
    
    
    img = Image.open(x) 
    img = img.resize((IMG_H, IMG_W), Image.ANTIALIAS) 
    img = ImageTk.PhotoImage(img) 
    panel = Label(root, image = img) 
    panel.image = img 
    panel.place(x=100, y=60)
    
def zoom_in():
    global IMG_H,IMG_W
    IMG_H=300
    IMG_W=300
    open_img_zoom()

def zoom_out():
    global IMG_H,IMG_W
    IMG_H=250
    IMG_W=250
    open_img_zoom()

In [7]:
#stack of images for Previous and Next Image
from tkinter import messagebox

def open_img_stack():
    global root,x
    img = Image.open(x) 
    img = img.resize((IMG_H, IMG_W), Image.ANTIALIAS) 
    img = ImageTk.PhotoImage(img) 
    panel = Label(root, image = img) 
    panel.image = img 
    panel.place(x=100, y=60)
    
def previous_img():
    global img_lcn_arr,x,count_img
    if(count_img==0):
        messagebox.showinfo("Error", "No Previous Image!!!")
    else:
        count_img=count_img-1
        if count_img!=0:
            x=img_lcn_arr[count_img-1]
            open_img_stack()
        else:
            messagebox.showinfo("Error", "No Previous Image!!!")

def next_img():
    global img_lcn_arr,x,count_img
    if(count_img>=len(img_lcn_arr)):
        messagebox.showinfo("Error", "No Next Image!!!")
    else:
        x=img_lcn_arr[count_img]
        open_img_stack()
        count_img=count_img+1


In [8]:
#Button Area

def function(root):
    
    count_pady=60
    btn = Button(root, text = 'New Image',command = open_img,width=15)
    btn.place(x=650, y=count_pady)
    
    count_pady=count_pady+40
    btn7 = Button(root, text = 'Previous Image',command = previous_img,width=15)
    btn7.place(x=650, y=count_pady)
    
    count_pady=count_pady+40
    btn2 = Button(root, text = 'Next Image',command = next_img,width=15)
    btn2.place(x=650, y=count_pady)
    
    count_pady=count_pady+40
    btn3 = Button(root, text = 'Generate Caption',command = generate_caption,width=15)
    btn3.place(x=650, y=count_pady)
    
    count_pady=count_pady+40
    btn4 = Button(root, text = 'Delete Caption',command =delete_caption,width=15)
    btn4.place(x=650, y=count_pady)
    
    btn5 = Button(root, text = 'Exit',command = root.destroy,width=15)
    btn5.place(x=650, y=400)
    
    count_pady=count_pady+40
    btn6 = Button(root, text = 'Zoom In',command = zoom_in,width=15)
    btn6.place(x=650, y=count_pady)
    
    count_pady=count_pady+40
    btn7 = Button(root, text = 'Zoom Out',command = zoom_out,width=15)
    btn7.place(x=650, y=count_pady)
    
    
    return root

In [9]:
#Import necessary library

import tkinter
from tkinter import *
from tkinter import filedialog
from PIL import ImageTk, Image  
import matplotlib.image as mpimg

In [10]:
#Menu Area
def menu_bar(root):
    menubar = Menu(root)
    
    # Adding File Menu and commands
    file = Menu(menubar, tearoff = 0)
    menubar.add_cascade(label ='File', menu = file)
    file.add_command(label ='New File', command = open_img)
    file.add_command(label ='Open File', command = open_img)
    file.add_separator()
    file.add_command(label ='Previous File', command = previous_img)
    file.add_command(label ='Next File', command = next_img)
    file.add_separator()
    file.add_command(label ='Exit', command = root.destroy)

    # Adding Edit Menu and commands
    edit = Menu(menubar, tearoff = 0)
    menubar.add_cascade(label ='Edit', menu = edit)
    edit.add_command(label ='Generate Caption', command = generate_caption)
    edit.add_command(label ='Delete Caption', command = delete_caption)
    edit.add_separator()
    edit.add_command(label ='Zoom In', command = zoom_in)
    edit.add_command(label ='Zoom Out', command = zoom_out)
    

    # Adding Help Menu
    help_ = Menu(menubar, tearoff = 0)
    menubar.add_cascade(label ='Help', menu = help_)
    help_.add_command(label ='Model Used', command = None)
    help_.add_command(label ='Dataset', command = None)
    help_.add_command(label ='Demo', command = None)
    help_.add_command(label ='Licence', command = None)
    help_.add_separator()
    help_.add_command(label ='About Tk', command = None)
    root.config(menu = menubar)
    
    return root

In [11]:
#Main GUI 
global root
def main():
    global root,IMG_H,IMG_W,img_lcn_arr,count_img
    IMG_H=250
    IMG_W=250
    img_lcn_arr=[]
    count_img=0
    root = tkinter.Tk()
    root.title('Image Captioner')
    root.geometry("800x500")
    root.resizable(width = False, height = False)
    l2=Label(root,text="Image Caption Generator",font='bold 12')
    l2.pack(side=TOP,pady=10)
    root=menu_bar(root)
    root=function(root)
    root.mainloop()
main()

two people are playing in the water
